In [1]:
import re
import warnings
import random
import numpy as np
import pandas as pd
import pymorphy3
from tqdm import tqdm
from corus import load_lenta

from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
from umap import UMAP
import hdbscan
from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

import nltk
from nltk.corpus import stopwords

warnings.filterwarnings('ignore')
tqdm.pandas()

RANDOM_STATE = 42
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

## 1. Загрузка и предобработка данных

In [2]:
# !curl -L https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz -o lenta-ru-news.csv.gz

In [3]:
records = load_lenta('lenta-ru-news.csv.gz')
data = pd.DataFrame(records)
data.columns = ['url','title','text','topic','tags','date']
data = data[['title','text']].sample(n=50000, random_state=RANDOM_STATE)
texts = (data['title'] + ' ' + data['text']).tolist()

In [4]:
# 2. Предобработка текстов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
morph = pymorphy3.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\verai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Для тематического моделирования ключевыми являются лемматизация и удаление шумовых токенов (запятых, цифр, стоп-слов), чтобы алгоритм не выделял темы по служебным словам. также возможна необходима фильтрация по длине документа, приведение к нижнему регистру, удаление HTML-артефактов, эмодзи и т. п.

Реализуем функцию предобработки текстов. В данной функции:
- Приводим текст к нижнему регистру.
- Удаляем HTML-теги.
- Оставляем только символы русского и английского алфавитов и пробелы.
- Производим токенизацию и удаляем стоп-слова.
- Выполняем лемматизацию с помощью pymorphy3.

Базовая классическая предобработка, помогает унифицировать представление текста, снизить размерность пространства признаков за счет сведения различных форм слова к единой норме и уменьшить шум. Остановилась на ней, так как лемматизация может давать потенциально лучше результат, остальные методы довольно базовые, не занимают много времени на обработку.

In [5]:
def preprocess(text):
    # Нижний регистр
    text = text.lower()
    # Удаление HTML-тегов
    text = re.sub(r'<.*?>', ' ', text)
    # Удаление всех символов, кроме букв и пробелов
    text = re.sub(r'[^a-zа-яё\s]', ' ', text)
    # Токенизация и удаление стоп-слов
    tokens = [w for w in text.split() if w not in stop_words]
    # Лемматизация
    lemmas = [morph.parse(w)[0].normal_form for w in tokens]
    return ' '.join(lemmas)

In [6]:
clean_texts = [preprocess(doc) for doc in tqdm(texts, desc='Preprocessing')]

Preprocessing: 100%|██████████| 50000/50000 [07:06<00:00, 117.13it/s]


## 2. Определяем компоненты пайплайна и обучение BERTopic

In [7]:
#  Encoder: sentence-transformers
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Быстрая модель с хорошим балансом качества и скорости

Обычно UMAP (или t-SNE, PCA) для понижения размерности эмбеддингов перед кластеризацией. UMAP хорошо сохраняет глобальную и локальную структуру, важны гиперпараметры n_neighbors, min_dist для контроля расстояний между документами.

In [8]:
# UMAP для снижения размерности перед кластеризацией
umap_model = UMAP(
    n_neighbors=15,      # баланс локального/глобального
    n_components=5,       # небольшой размер для кластеров, позже можно визуализировать через 2D-преобразование, хватит для сохранении информации
    min_dist=0.1,         # плотное размещение точек, но тк тексты разные возьму больше 0
    metric="cosine", 
    random_state=RANDOM_STATE
)

HDBSCAN — иерархический плотностный алгоритм, умеющий выделять документы без явной тематики, +не требует количества кластеров

In [9]:
# HDBSCAN для кластеризации
hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=50,      # минимальный размер темы, не берем слишком маленькие группы
    metric='euclidean',       # расстояние в embedding-пространстве
    prediction_data=True
)

CountVectorizer:
- ngram_range=(1,2): учитывает униграммы и биграммы,
- min_df=5: исключает слова, встречающиеся менее чем в 5 документах,
- max_df=0.9: исключает слишком частые слова (более чем в 90% документов).

Позволяет улучшить качество тем, отсекая редкие шумовые и слишком общие токены.

In [10]:
vectorizer_model = CountVectorizer(ngram_range=(1, 2), min_df=5, max_df=0.9)

ClassTFIDFTransformer:
- reduce_frequent_words=True: дополнительно снижает вес часто встречающихся токенов,
Для извлечения более информативных топ-токенов для каждой темы.

In [12]:
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True,)

In [13]:
# Инициализация и обучение BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    language="russian",
    calculate_probabilities=True, 
    verbose=True,
)

In [14]:
topics, probabilities = topic_model.fit_transform(clean_texts)

2025-05-01 12:02:09,364 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

2025-05-01 12:03:02,831 - BERTopic - Embedding - Completed ✓
2025-05-01 12:03:02,832 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-01 12:03:46,266 - BERTopic - Dimensionality - Completed ✓
2025-05-01 12:03:46,267 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-01 12:03:52,502 - BERTopic - Cluster - Completed ✓
2025-05-01 12:03:52,509 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-01 12:04:02,162 - BERTopic - Representation - Completed ✓


## 3. Визуализация результатов

In [15]:
# Топ-токены для каждой темы
topic_info = topic_model.get_topic_info()
topic_info.head()  # наиболее важные темы

,Topic,Count,Name,Representation,Representative_Docs
0,-1,10079,-1_самолёт_учёный_матч_рубль,"[самолёт, учёный, матч, рубль, погибнуть, of, ...",[судья бальтасар гарсон завести дело отношение...
1,0,21191,0_украина_рубль_риа_риа новость,"[украина, рубль, риа, риа новость, задержать, ...",[москва рассказать желание запад напугать евро...
2,1,3193,1_цска_цб_церковь_банк,"[цска, цб, церковь, банк, цру, центробанк, цик...",[цска зенит сыграть вничью забить шесть мяч че...
3,2,2818,2_школа_шахта_школьник_шарапов,"[школа, шахта, школьник, шарапов, ученик, ребё...",[шестилетний россиянин победить шоу талант кит...
4,3,2579,3_мэр_аэропорт_венесуэла_аэрофлот,"[мэр, аэропорт, венесуэла, аэрофлот, аэс, рао,...",[росэнергоатом ростелеком создать крупный дата...


In [16]:
#количество тем
len(topic_model.get_topics()) - 1

41

In [18]:
# Список топ токенов для темы #0
print("Топ-токены темы 1:", topic_model.get_topic(0))

Топ-токены темы 1: [('украина', 0.12736140823997968), ('рубль', 0.12617583570491211), ('риа', 0.12009068702217965), ('риа новость', 0.1199587344119356), ('задержать', 0.11701731924845428), ('уголовный', 0.11480504590794754), ('путин', 0.11469298130600754), ('депутат', 0.11271316563716445), ('республика', 0.10988583316732341), ('орган', 0.10937164885992962)]


In [ ]:
# 2D визуализация документов
fig1 = topic_model.visualize_documents(clean_texts)
fig1.show()

In [ ]:
# Распределение тем по токенам для заданного документа
doc_id = 17
print(clean_texts[doc_id])
fig_dist = topic_model.visualize_distribution(probabilities[doc_id], min_probability=0.015)
fig_dist.show()

## 4. Оценка качества тем

### 4.1 Topic Diversity

In [28]:
def topic_diversity(model, top_n=10):
    '''Доля уникальных токенов среди всех топ-N токенов.'''
    topics = model.get_topics()
    all_tokens = []
    for _, tokens in topics.items():
        all_tokens.extend([t for t, _ in tokens[:top_n]])
    unique_count = len(set(all_tokens))
    total_count = len(all_tokens)
    return unique_count / total_count

In [34]:
td = topic_diversity(topic_model, top_n=10)
print('topic_diversity:', td)

topic_diversity: 0.9142857142857143


### 4.2 UMass Coherence через Gensim

In [31]:
# Для этого необходимо подготовить корпус токенов
tokenized_texts = [doc.split() for doc in clean_texts]

dictionary = Dictionary(tokenized_texts)
corpus_gensim = [dictionary.doc2bow(text) for text in tokenized_texts]

In [32]:
def compute_umass_coherence(model, texts, dictionary, corpus, top_n=10):
    '''Вычисляет UMass coherence для каждой темы и среднее значение.'''
    topics = [ [w for w, _ in model.get_topic(t)[:top_n]]
               for t in model.get_topic_info().Topic if t != -1 ]
    cm = CoherenceModel(
        topics=topics,
        texts=texts,
        dictionary=dictionary,
        corpus=corpus,
        coherence='u_mass'
    )
    return cm.get_coherence()

In [33]:
umass_score = compute_umass_coherence(topic_model, tokenized_texts, dictionary, corpus_gensim)
print(f"UMass Coherence: {umass_score:.4f}")

UMass Coherence: -4.5876


### Выводы
- **Topic Diversity ≈ 0.91**  
  Темы получились разнородными — 90 % токенов в топ-10 уникальны.

- **UMass Coherence ≈ –4.6**  
  Стабильный, но средний уровень когерентности; есть «размытые» темы.

---

Что можно исправить

1. Поэкспериментировать с HDBSCAN  
   – увеличить cluster_selection_epsilon для слияния близких тем  
   – скорректировать min_samples для жёсткой фильтрации шума  
2. добавить биграммы/триграммы для векторизации (ngram_range=(1,3)) или POS-фильтрацию  
3. Эмбеддинги– протестировать другие Эмбеддинги, например RuBERT-модели 
4. UMAP – увеличить n_components (10–15) перед кластеризацией